# CrewAI Bedrock Agent Core Memory Demo

This notebook demonstrates the core save and search functionality of AWS Bedrock Agent Core memory integration with CrewAI.

## Features

- Saving information to Agent Core memory
- Searching and retrieving stored information
- Memory enhancing conversational context
- Real-world conversation scenarios with memory persistence

## Prerequisites

### 1. AWS Setup
- AWS credentials configured (`aws configure` or environment variables)
- AWS Bedrock Agent Core Memory resource created
- Required IAM permissions:
  - `bedrock-agentcore:CreateEvent`
  - `bedrock-agentcore:ListEvents` 
  - `bedrock-agentcore:RetrieveMemories`
  - `bedrock-agentcore-control:CreateMemory` -> If you don't have a memory resource pre-configured
  - `bedrock-agentcore-control:GetMemory` -> If you don't have a memory resource pre-configured



### 2. Dependencies
```bash
pip install crewai[agentcore] boto3
```

## 1. Initial Setup and Configuration

In [1]:
# Import required libraries
from crewai.memory.external.external_memory import ExternalMemory
from crewai.memory.storage.bedrock_agentcore_storage import (
    BedrockAgentCoreConfig,
    BedrockAgentCoreStorage,
)
from crewai import Agent, Task, Crew, LLM, Process

import boto3
from botocore.exceptions import ClientError

import time
from typing import Any, Dict, List, Optional
import uuid


print("✅ Libraries imported successfully")

In [2]:
# Configuration - Update these with your actual values
MEMORY_NAME = "SaveSearchDemo"
ACTOR_ID = "demo-user-456"  # Unique identifier for the user
SESSION_ID = "save-search-session"  # Unique session identifier
REGION = "us-west-2"  # AWS region

print("Configuration:")
print(f"  Memory Name: {MEMORY_NAME}")
print(f"  Actor ID: {ACTOR_ID}")
print(f"  Session ID: {SESSION_ID}")
print(f"  Region: {REGION}")

In [3]:
# Verify AWS credentials
try:
    session = boto3.Session()
    credentials = session.get_credentials()
    if credentials is None:
        raise Exception("No AWS credentials found")

    # Test basic access
    sts = session.client("sts")
    identity = sts.get_caller_identity()
    print(f"✅ AWS credentials verified for account: {identity.get('Account')}")

except Exception as e:
    print(f"❌ AWS credentials issue: {e}")
    print("Please configure your AWS credentials before proceeding.")

## 2. Create Agent Core Memory Storage

We'll create a memory instance with conversation and preference strategies for enhanced functionality.

In [ ]:
bedrock_control_client = boto3.client("bedrock-agentcore-control", region_name=REGION)


def create_memory_and_wait(
    name: str,
    strategies: List[Dict[str, Any]],
    description: Optional[str] = None,
    event_expiry_days: int = 90,
    memory_execution_role_arn: Optional[str] = None,
    max_wait: int = 400,
    poll_interval: int = 10,
) -> Dict[str, Any]:
    """Create a memory and wait for it to become ACTIVE.

    This method creates a memory and polls until it reaches ACTIVE status,
    providing a convenient way to ensure the memory is ready for use.

    Args:
        name: Name for the memory resource
        strategies: List of strategy configurations
        description: Optional description
        event_expiry_days: How long to retain events (default: 90 days)
        memory_execution_role_arn: IAM role ARN for memory execution
        max_wait: Maximum seconds to wait (default: 300)
        poll_interval: Seconds between status checks (default: 10)

    Returns:
        Created memory object in ACTIVE status

    Raises:
        TimeoutError: If memory doesn't become ACTIVE within max_wait
        RuntimeError: If memory creation fails
    """
    print(f"Creating Memory with name: {name} and {len(strategies)} strategies")

    params = {
        "name": name,
        "eventExpiryDuration": event_expiry_days,
        "memoryStrategies": strategies,
        "clientToken": str(uuid.uuid4()),
    }

    if description is not None:
        params["description"] = description

    if memory_execution_role_arn is not None:
        params["memoryExecutionRoleArn"] = memory_execution_role_arn

    # Create the memory
    memory = bedrock_control_client.create_memory(**params)

    print(memory)

    memory_id = memory["memory"]["id"]
    if memory_id is None:
        memory_id = ""
    print(f"✅ Created memory {memory_id}, waiting for ACTIVE status...")

    start_time = time.time()
    while time.time() - start_time < max_wait:
        elapsed = int(time.time() - start_time)

        try:
            response = bedrock_control_client.get_memory(memoryId=memory_id)

            if response["memory"]["status"] == "ACTIVE":
                print(f"Memory {memory_id} is now ACTIVE (took {elapsed} seconds)")
                return response["memory"]
            elif response["memory"]["status"] == "FAILED":
                # Get failure reason if available
                failure_reason = response["memory"].get("failureReason", "Unknown")
                raise RuntimeError("Memory creation failed: %s" % failure_reason)
            else:
                print(
                    f"Memory status: {response['memory']['status']} ({elapsed} seconds elapsed)"
                )

        except ClientError as e:
            print(f"❌ Error checking memory status: {e}")
            raise

        time.sleep(poll_interval)

    raise TimeoutError(
        f"Memory {memory_id} did not become ACTIVE within {max_wait} seconds"
    )

In [ ]:
# Create Memory using boto3 client
# Note: Skip if you already have a Bedrock Agent Core Memory resource created
# Create memory with conversation and preference strategies
try:
    response = create_memory_and_wait(
        name=MEMORY_NAME,
        strategies=[
            {
                "summaryMemoryStrategy": {
                    "name": "ConversationSummarizer",
                    "namespaces": ["/conversations/{actorId}/{sessionId}"],
                }
            },
            {
                "semanticMemoryStrategy": {
                    "name": "SemanticMemory",
                    "namespaces": ["/preferences/{actorId}"],
                }
            },
            {
                "userPreferenceMemoryStrategy": {
                    "name": "UserPreferences",
                    "namespaces": ["/strategies/{memoryStrategyId}/actors/{actorId}"],
                }
            },
        ],
    )
    print("✅ Memory Creation Complete")
except Exception as e:
    print(f"❌ Error creating memory: {e}")
    print(
        "Note: If you already have a memory resource, you can skip this step and set the variables manually."
    )

In [ ]:
# Extract required variables from response
MEMORY_ID = response["id"]
SUMMARY_STRATEGY_ID = response["strategies"][0]["strategyId"]
SEMANTIC_STRATEGY_ID = response["strategies"][1]["strategyId"]
USER_PREFERENCE_STRATEGY_ID = response["strategies"][2]["strategyId"]

# Note: Uncomment below if you already have a memory resource created:
# MEMORY_ID = "memory-id"                                           # Replace with actual ID
# SUMMARY_STRATEGY_ID = "summary-strategy-id"                       # Replace with actual ID
# SEMANTIC_STRATEGY_ID = "semantic-strategy-id"                     # Replace with actual ID
# USER_PREFERENCE_STRATEGY_ID = "user-preference-strategy-id"       # Replace with actual ID

print(f"Memory ID: {MEMORY_ID}")
print(f"Conversation Strategy ID: {SUMMARY_STRATEGY_ID}")
print(f"Preference Strategy ID: {SEMANTIC_STRATEGY_ID}")
print(f"Preference Strategy ID: {USER_PREFERENCE_STRATEGY_ID}")

In [4]:
# Create config for Bedrock AgentCore external memory
config = BedrockAgentCoreConfig(
    memory_id=MEMORY_ID,
    actor_id=ACTOR_ID,
    session_id=SESSION_ID,
    region_name=REGION,
    # namespaces to be searched through
    namespaces=[
        BedrockAgentCoreStorage.resolve_namespace_template(
            "/conversations/{actorId}/{sessionId}",
            ACTOR_ID,
            SESSION_ID,
            SUMMARY_STRATEGY_ID,
        ),
        BedrockAgentCoreStorage.resolve_namespace_template(
            "/preferences/{actorId}",
            ACTOR_ID,
            SESSION_ID,
            SEMANTIC_STRATEGY_ID,
        ),
        BedrockAgentCoreStorage.resolve_namespace_template(
            "/strategies/{memoryStrategyId}/actors/{actorId}",
            ACTOR_ID,
            SESSION_ID,
            USER_PREFERENCE_STRATEGY_ID,
        ),
    ],
)

# Create external memory instance
memory = ExternalMemory(
    embedder_config={
        "provider": "agentcore",
        "config": config,
    }
)

print("Agent Core memory storage created and initialized")
print("   Provider: agentcore")
print(f"   Memory ID: {MEMORY_ID}")
print(f"   Actor: {ACTOR_ID}")
print(f"   Session: {SESSION_ID}")

In [ ]:
# Create a simple crew to initialize memory
llm = LLM(
    model="bedrock/anthropic.claude-3-5-sonnet-20241022-v2:0", region_name="us-west-2"
)

memory_agent = Agent(
    role="Memory Assistant",
    goal="Answer the user's question utilizing memory if present.",
    backstory="""You are a helpful assistant that answers the user's questions using memory if present""",
    verbose=False,
    allow_delegation=False,
    llm=llm,
)

memory_task = Task(
    description="Answer the user's question: {question}.",
    agent=memory_agent,
    expected_output="Answer.",
)

# Initialize crew with memory
crew = Crew(
    agents=[memory_agent],
    tasks=[memory_task],
    process=Process.sequential,
    verbose=False,
    external_memory=memory,
)

# Set crew for memory management
memory.set_crew(crew)

print("✅ Memory assistant crew created and initialized")

## 5. Simulate Realistic User Queries

Now let's simulate realistic user queries that would benefit from memory context, similar to the conversation messages in the interactive test.

In [ ]:
def ask_question(question: str):
    """Ask a question and get a memory-enhanced response"""
    print(f"❓ User Question: {question}")
    print("=" * 60)

    result = crew.kickoff({"question": question})

    print("\n🤖 AI Response:")
    print(result.raw)
    print("\n" + "=" * 80 + "\n")

    return result

In [ ]:
# Query 1: No Memory, trigger inserts
ask_question(
    "Hi, I'm Sarah, a software engineer learning about AI. I prefer detailed technical explanations. I like Python and Java - what are good projects to get started?"
)

# Sample Crew Output:
# 🤖 AI Response:
# Hi Sarah! As a software engineer with Python and Java experience, here are some detailed technical project recommendations to start your AI journey:

# 1. Machine Learning Fundamentals Projects:
# - Build a Neural Network from Scratch (Python)
#   * Implement backpropagation manually using NumPy
#   * Create a simple feed-forward network for MNIST classification
#   * Learn core concepts like gradient descent and activation functions

# 2. Natural Language Processing Projects:
# - Text Classification System (Python)
#   * Use NLTK or spaCy for text preprocessing
#   * Implement TF-IDF vectorization
#   * Build sentiment analysis using scikit-learn
# - Chatbot Implementation (Java)
#   * Use Stanford NLP library
#   * Implement pattern matching and response generation
#   * Add basic context handling

# 3. Computer Vision Projects:
# - Image Classification (Python)
#   * Use TensorFlow/Keras or PyTorch
#   * Implement CNN architecture
#   * Work with datasets like CIFAR-10
# - Object Detection System (Java)
#   * Use DJL (Deep Java Library)
#   * Implement YOLO or SSD algorithms
#   * Real-time webcam integration

# 4. Reinforcement Learning:
# - Game AI (Python)
#   * Implement Q-learning for simple games
#   * Use OpenAI Gym environments
#   * Create self-learning agents

# Technical Framework Recommendations:
# - Python Libraries:
#   * NumPy for numerical computations
#   * Pandas for data manipulation
#   * scikit-learn for traditional ML
#   * TensorFlow/PyTorch for deep learning
# - Java Frameworks:
#   * DJL for deep learning
#   * Weka for traditional ML
#   * Deeplearning4j for neural networks

# Development Tips:
# 1. Use version control (Git) from the start
# 2. Implement proper logging and testing
# 3. Focus on clean code architecture
# 4. Document your model architectures
# 5. Use proper data validation techniques

# Start with smaller projects and gradually increase complexity. Each project should include:
# - Data preprocessing pipeline
# - Model implementation
# - Evaluation metrics
# - Error analysis
# - Performance optimization

# Remember to focus on understanding the mathematical concepts behind each algorithm you implement. This will give you a stronger foundation for more advanced AI projects.

In [ ]:
# Query 2: Trigger Short Term Memory Retrieve
ask_question("What do you know about the coding languages that I like?")


# Sample Crew Output
# 🤖 AI Response:
# Based on the memory available, I can see that you like Python and Java. Specifically, this information comes from a previous
# interaction where you introduced yourself as a software engineer learning about AI who has experience with these two programming languages.
# These are both excellent choices for software development and AI work - Python is widely used in AI/ML development due to its extensive libraries and frameworks,
# while Java offers robust enterprise-level application development capabilities.
# However, I don't have any additional memories about specific aspects of these languages that you particularly enjoy or other programming languages you might use.
# If you'd like to share more about what specifically draws you to Python and Java, or tell me about other languages you work with, I'd be happy to learn more about your programming preferences.

In [ ]:
# Query 3: Another Short Term Memory Retrieval
ask_question("Can you remind me what I mentioned?")

# Sample Response:
# 🤖 AI Response:
# Based on reviewing the memories, I can remind you of a few key things that were mentioned:

# 1. You introduced yourself as Sarah, a software engineer who is learning about AI
# 2. You mentioned that you prefer detailed technical explanations
# 3. You shared that you like Python and Java programming languages
# 4. You asked about good projects to get started with AI using these languages, to which I provided an extensive technical response with project recommendations including:
#    - Machine Learning Fundamentals Projects
#    - Natural Language Processing Projects
#    - Computer Vision Projects
#    - Reinforcement Learning Projects
#    Along with detailed framework recommendations and development tips

# These are all the specific things that were mentioned in our previous interactions based on the available memories. Would you like to know more about any of these aspects in particular?

In [ ]:
time.sleep(120)  # wait 2 min for Long Term Memory retrieval to complete

In [ ]:
# # Query 4: Same as Query 3 but slightly different response due to Long Term Memory Retrieval
ask_question("Can you remind me what I mentioned?")

# # Sample Response:
# # 🤖 AI Response:
# Yes, I can remind you of what you've mentioned in our previous interactions! You shared that:

# 1. You are a software engineer who is learning about AI
# 2. You have experience with two programming languages:
#    - Python
#    - Java
# 3. You prefer receiving detailed technical explanations
# 4. You previously asked about coding languages, specifically what I knew about your language preferences

# We also had a detailed discussion about AI project recommendations for your skill set, where I provided you with specific project suggestions including:
# - Machine Learning Fundamentals Projects (Neural Network from Scratch in Python)
# - Natural Language Processing Projects (Text Classification System in Python and Chatbot Implementation in Java)
# - Computer Vision Projects (Image Classification in Python and Object Detection System in Java)
# - Reinforcement Learning Projects (Game AI in Python)

# These recommendations were tailored specifically to your background as a software engineer and your experience with Python and Java.